### Plot de Geolocalização utilizando coordenadas em Json

#### Instalação do pacote folium
sudo python -m pip install folium

In [61]:
import pandas as pd

# Definindo JSON de forma manual
end_turma ="""
        {
            "Minion":{
                     "0":"Stuart",
                     "1":"Kevin",
                     "2":"Bob",
                     "3":"Dave"
                         },
            "Latitude":{
                    "0":-23.7273345,
                    "1":-23.6273344,
                    "2":-23.6344377,
                    "3":-23.6344379
                    
                        },
            "Longitude":{
                    "0":-46.7369416,
                    "1":-46.6369415,
                    "2":-46.6369419,
                    "3":-46.7246571
                         }
        }"""

# Traduzindo JSON para um dataframe (pandas)
df = pd.read_json(end_turma)

print('Dataframe gerado pelo JSON')
df.head(10)

,Latitude,Longitude,Minion
0,-23.727335,-46.736942,Stuart
1,-23.627334,-46.636941,Kevin
2,-23.634438,-46.636942,Bob
3,-23.634438,-46.724657,Dave


In [62]:
# Vamos colocar os endereços com latitude e longitude no Mapa
# http://www.mapcoordinates.net/pt

import folium

location = df['Latitude'].mean(), df['Longitude'].mean()
locationlist = df[["Latitude","Longitude"]].values.tolist()
labels = df["Minion"].values.tolist()

m = folium.Map(location=location, zoom_start=10)
for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(m)
    
 
    
m

### Exemplo Kaggle
https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales/downloads/accidents_2005_to_2007.csv/10

In [14]:
%run -i '/home/hadoop/.Turing/TuringAllFunctions.py'

------------- Turing Lab Data Science --------------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 04/05/2019 -------------------------------
----------------------------------------------------------
--- Consulte o manual de Biblioteca de Funções Turing: ---


----------------------------------------------------------


In [15]:
#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-treinamentos'
id_empresa = '1022' 

#---------- Gerando Credenciais para Acesso Seguro ao Lake ----------------------
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)

In [16]:
# Lendo arquivo CSV

nm_file = 'accidents_2005_to_2007.csv'
delimiter = ',' 
path_file = 'Treinamento/Dados/'+str(nm_file)

df_00 = TuringReadS3CSVFile(nm_bucket,path_file,sep=',')
df_00.shape

(570011, 33)

In [45]:
df_00.head()

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,1,2,1,1,04/01/2005,...,Zebra crossing,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,1,Yes,E01002849,2005
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,1,3,1,1,05/01/2005,...,Pedestrian phase at traffic signal junction,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01002909,2005
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,1,3,2,1,06/01/2005,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01002857,2005
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,1,3,1,1,07/01/2005,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01002840,2005
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,1,3,1,1,10/01/2005,...,No physical crossing within 50 meters,Darkness: Street lighting unknown,Fine without high winds,Wet/Damp,None,None,1,Yes,E01002863,2005


In [70]:
# Ensure you're handing it floats
df_00['Latitude'] = df_00['Latitude'].astype(float)
df_00['Longitude'] = df_00['Longitude'].astype(float)

# Gerando amostra aleatoria para plotar icones no mapa
heat_df = TuringSampleData(df_00,pct=0.001,seed=12345,n_obs=0)

# removendo valores com missings
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude','Road_Surface_Conditions'])

location = heat_df['Latitude'].mean(), heat_df['Longitude'].mean()
locationlist = heat_df[["Latitude","Longitude"]].values.tolist()
labels = heat_df["Road_Surface_Conditions"].values.tolist()
heat_df.shape


(569, 33)

In [71]:
m = folium.Map(location=location, zoom_start=10)
for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(m)  
 
    
m